In [ ]:
# |default_exp chroma_db

## Install dependencies

## Make an app with Gradio

In [ ]:
# |export
import chromadb
from chromadb.config import Settings
from dotenv import load_dotenv
import os

This is a simple RAG chatbot built on top of Llama Index and Gradio. It allows you to upload any text or PDF files and ask questions about them!
Before running this, make sure you have exported your OpenAI API key as an environment variable:

```bash
export OPENAI_API_KEY="mykey"
```

In [ ]:
# |export
load_dotenv()

In [ ]:
oai_key = os.getenv('GEMINI_API_KEY')
print(oai_key)

In [ ]:
# |export
# client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet",persist_directory="db/"))
client = chromadb.PersistentClient(path="../db")


In [ ]:
# |export
collection = client.create_collection(name="Students")

In [ ]:
# |export
student_info = """
Alexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,
is a member of the programming and chess clubs who enjoys pizza, swimming, and hiking
in her free time in hopes of working at a tech company after graduating from the University of Washington.
"""

club_info = """
The university chess club provides an outlet for students to come together and enjoy playing
the classic strategy game of chess. Members of all skill levels are welcome, from beginners learning
the rules to experienced tournament players. The club typically meets a few times per week to play casual games,
participate in tournaments, analyze famous chess matches, and improve members' skills.
"""

university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

In [ ]:
# |export
collection.add(
    documents = [student_info, club_info, university_info],
    metadatas = [{"source": "student_info"}, {"source": "club_info"}, {"source": "university_info"}],
    ids = ["id1", "id2", "id3"]
)

In [ ]:
#|export
results = collection.query(
    query_texts=["What is the student name?"],
    n_results=2
)

In [ ]:
results

In [ ]:
# |export
demo.launch()

In [ ]:
# this is only necessary in a notebook
demo.close()

## Create a `requirements.txt` file

In [ ]:
%%writefile ../requirements.txt
fastcore

## Convert this notebook into a Gradio app

In [ ]:
# from nbdev.export import nb_export
# nb_export('01_gradio.ipynb', lib_path='.', name='gradio')

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()